In [1]:
import argparse
import datetime
import os
from os.path import basename, dirname, join

import glob

import numpy as np
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
import matplotlib.pyplot as plt
import random

from tensorflow.keras.layers.experimental.preprocessing import Resizing 
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

from archive import Archive
from data_generator import DataGeneratorFrom_npz_File, DataGenerator
from utility import Configure, between_zero_and_one_float_type
from train_model import FileBasedConfigure

from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *

from netCDF4 import Dataset

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-mrh9ma29 because the default path (/.config/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weigths = tf.expand_dims(tf.Variable(weights, dtype=tf.float32), axis=0)
    weigths = tf.expand_dims(weights, axis=0)
    
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= tf.reduce_sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        # calc
        loss = y_true * tf.math.log(y_pred) * weights
        loss = -tf.reduce_sum(loss, -1)
        return loss
    
    return loss

In [13]:
class_weight = np.array([0.5, 0.7, 0.8])

weights = np.expand_dims(class_weight/np.sum(class_weight),axis=0)

weighted_loss = weighted_categorical_crossentropy(weights)

In [14]:
idir = '/input_data/exp_data/binary_output'
train_ratio = 0.8
npz_files = sorted(glob.glob(f'{idir}/*npz'))
random.shuffle(npz_files)
train_files_number = int(len(npz_files) * train_ratio)
train_files = npz_files[:train_files_number]
valid_files = npz_files[train_files_number:]

input_var_names = ['nersc_sar_primary', 'nersc_sar_secondary']
amsr2_var_names = [
    'btemp_6_9h',
    'btemp_6_9v',
    'btemp_7_3h',
    'btemp_7_3v',
    'btemp_10_7h',
    'btemp_10_7v',
    'btemp_18_7h',
    'btemp_18_7v',
    'btemp_23_8h',
    'btemp_23_8v',
    'btemp_36_5h',
    'btemp_36_5v',
    'btemp_89_0h',
    'btemp_89_0v',
 ]
output_var_name = 'ice_type'

dims_input = np.load(npz_files[0])[input_var_names[0]].shape
dims_output = np.load(npz_files[0])[output_var_name].shape
dims_amsr2 = np.load(npz_files[0])[amsr2_var_names[0]].shape



batch_size_list = [20,40,80]
list_filters = [8, 16, 32]
learning_rate = [1e-3, 1e-4, 1e-5]
conv_number = [1,2]
unet_depth = [3,4]
losses = ['mse', 'mae', 'categorical_crossentropy', weighted_loss]

from keras.callbacks import CSVLogger

csv_logger = CSVLogger('log.csv', append=True, separator=';')
val_loss_cb = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [15]:
def train_val_generator(batch_size):
    params = {'dims_input':      (*dims_input, len(input_var_names)),
                       'dims_output':     (*dims_output,),
                       'dims_amsr2':      (*dims_amsr2, len(amsr2_var_names)),
                       'output_var_name': output_var_name,
                       'input_var_names': input_var_names,
                       'amsr2_var_names': amsr2_var_names,
                       'batch_size':      batch_size,
                       'shuffle_on_epoch_end': False,
                       }
    input_size = params['dims_input']
    amsr2_size = params['dims_amsr2']


    training_generator = DataGeneratorFrom_npz_File(train_files, **params)
    validation_generator = DataGeneratorFrom_npz_File(valid_files, **params)

    return validation_generator, training_generator, input_size, amsr2_size

In [16]:
params_list = []
for i in range(len(losses)):
    
    combination = [batch_size_list[0], list_filters[0], learning_rate[1], conv_number[1], unet_depth[0], losses[3]]
    params_list.append(combination)
    combination = [batch_size_list[0], list_filters[1], learning_rate[1], conv_number[0], unet_depth[0], losses[3]]
    params_list.append(combination)
    
    
params_list

[[20,
  8,
  0.0001,
  2,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  16,
  0.0001,
  1,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  8,
  0.0001,
  2,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  16,
  0.0001,
  1,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  8,
  0.0001,
  2,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  16,
  0.0001,
  1,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  8,
  0.0001,
  2,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>],
 [20,
  16,
  0.0001,
  1,
  3,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>]]

# U-net DEPTH = 3

In [17]:
# U-net DEPTH = 3
for batch_size, n_filters, rate, cnumber, depth, loss_ in params_list:
    params = {'dims_input':      (*dims_input, len(input_var_names)),
           'dims_output':     (*dims_output,),
           'dims_amsr2':      (*dims_amsr2, len(amsr2_var_names)),
           'output_var_name': output_var_name,
           'input_var_names': input_var_names,
           'amsr2_var_names': amsr2_var_names,
           'batch_size':      batch_size,
           'shuffle_on_epoch_end': False,
           }


    training_generator = train_val_generator(batch_size)[0]
    validation_generator = train_val_generator(batch_size)[1]
    input_size = train_val_generator(batch_size)[2]
    amsr2_size = train_val_generator(batch_size)[3]

    
    inputs = layers.Input(shape=input_size)
    
    inputs_2 = layers.Input(shape=amsr2_size)


    conv2 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    if cnumber ==2:
        conv2 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)

    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    if cnumber ==2:
        conv3 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)

    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    if cnumber ==2:
        conv4 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)

    

    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

    amsr2_up = Resizing(32,32)(inputs_2)

    pool4 = layers.Concatenate(axis = 3)([pool4 , amsr2_up])

    conv5 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    if cnumber ==2:
        conv5 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

    


    up6 = layers.Conv2D(4*n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv5))

    merge6 = layers.Concatenate(axis = 3)([conv4,up6])

    conv6 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    if cnumber ==2:
        conv6 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = layers.Conv2D(2*n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))

    merge7 = layers.Concatenate(axis = 3)([conv3,up7])

    conv7 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    if cnumber ==2:
        conv7 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = layers.Conv2D(n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))

    merge8 = layers.Concatenate(axis = 3)([conv2,up8])

    conv8 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    if cnumber ==2:
        conv8 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    conv10 = layers.Conv2D(3, 1, activation = 'softmax')(conv8)

    model = Model(inputs = [inputs, inputs_2], outputs = conv10)

    model.compile(optimizer = Adam(learning_rate=1e-4), loss = loss_, metrics = ['accuracy'])
    model.summary()
    csv_logger = CSVLogger('binary_chosen_'+str(20)+'_'+str(n_filters)+'_'+ str(1e-4)+'_'+str(cnumber)+'_'+ str(depth)+'_'+str(loss_)+'_epochs=15'+'_'+'_model_2_log.csv', append=True, separator=';')
    model.fit(training_generator, validation_data = validation_generator, epochs = 1,callbacks= [ val_loss_cb])

    model.save('binary_chosen_'+str(20)+'_'+str(n_filters)+'_'+ str(1e-4)+'_'+str(cnumber)+'_'+ str(depth)+'_'+str(loss_)+'_epochs=15'+'_'+'_model_2')
    
   

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            [(None, 256, 256, 2) 0                                            
__________________________________________________________________________________________________
conv2d_51 (Conv2D)              (None, 256, 256, 8)  152         input_9[0][0]                    
__________________________________________________________________________________________________
conv2d_52 (Conv2D)              (None, 256, 256, 8)  584         conv2d_51[0][0]                  
__________________________________________________________________________________________________
max_pooling2d_12 (MaxPooling2D) (None, 128, 128, 8)  0           conv2d_52[0][0]                  
____________________________________________________________________________________________

KeyboardInterrupt: 

In [8]:
params_list = []
for i in range(len(losses)):
    
    combination = [batch_size_list[0], list_filters[0], learning_rate[1], conv_number[1], unet_depth[1], losses[i]]
    params_list.append(combination)
    
    
    
params_list

[[20, 8, 0.0001, 2, 4, 'mse'],
 [20, 8, 0.0001, 2, 4, 'mae'],
 [20, 8, 0.0001, 2, 4, 'categorical_crossentropy'],
 [20,
  8,
  0.0001,
  2,
  4,
  <function __main__.weighted_categorical_crossentropy.<locals>.loss(y_true, y_pred)>]]

# U-net DEPTH = 4

In [9]:
# U-net DEPTH = 4
for batch_size, n_filters, rate, cnumber, depth, loss_ in params_list:
    params = {'dims_input':      (*dims_input, len(input_var_names)),
           'dims_output':     (*dims_output,),
           'dims_amsr2':      (*dims_amsr2, len(amsr2_var_names)),
           'output_var_name': output_var_name,
           'input_var_names': input_var_names,
           'amsr2_var_names': amsr2_var_names,
           'batch_size':      batch_size,
           'shuffle_on_epoch_end': False,
           }


    training_generator = train_val_generator(batch_size)[0]
    validation_generator = train_val_generator(batch_size)[1]
    input_size = train_val_generator(batch_size)[2]
    amsr2_size = train_val_generator(batch_size)[3]

    
    inputs = layers.Input(shape=input_size)
    
    inputs_2 = layers.Input(shape=amsr2_size)

    conv1 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    if cnumber ==2:
        conv1 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)

    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    if cnumber ==2:
        conv2 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)

    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    if cnumber ==2:
        conv3 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)

    pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    if cnumber ==2:
        conv4 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)

    

    pool4 = layers.MaxPooling2D(pool_size=(2, 2))(conv4)

#     amsr2_up = Resizing(32,32)(inputs_2)

    pool4 = layers.Concatenate(axis = 3)([pool4 , inputs_2])

    conv5 = layers.Conv2D(16*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    if cnumber ==2:
        conv5 = layers.Conv2D(16*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)

    


    up6 = layers.Conv2D(8*n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv5))

    merge6 = layers.Concatenate(axis = 3)([conv4,up6])

    conv6 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    if cnumber ==2:
        conv6 = layers.Conv2D(8*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = layers.Conv2D(4*n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv6))

    merge7 = layers.Concatenate(axis = 3)([conv3,up7])

    conv7 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    if cnumber ==2:
        conv7 = layers.Conv2D(4*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = layers.Conv2D(2*n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv7))

    merge8 = layers.Concatenate(axis = 3)([conv2,up8])

    conv8 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    if cnumber ==2:
        conv8 = layers.Conv2D(2*n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = layers.Conv2D(n_filters, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(layers.UpSampling2D(size = (2,2))(conv8))

    merge9 = layers.Concatenate(axis = 3)([conv1,up9])

    conv9 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    if cnumber ==2:
        conv9 = layers.Conv2D(n_filters, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv9 = layers.Conv2D(4, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)

    conv10 = layers.Conv2D(4, 1, activation = 'softmax')(conv9)


    model = Model(inputs = [inputs, inputs_2], outputs = conv10)

    model.compile(optimizer = Adam(learning_rate=1e-4), loss = loss_, metrics = ['accuracy'])
    model.summary()
    csv_logger = CSVLogger('binary_chosen_'+str(20)+'_'+str(n_filters)+'_'+ str(1e-4)+'_'+str(cnumber)+'_'+ str(depth)+'_'+str(loss_)+'_epochs=15'+'_'+'_model_2_log.csv', append=True, separator=';')
    model.fit(training_generator, validation_data = validation_generator, epochs = 15,callbacks= [ val_loss_cb])

    model.save('binary_chosen_'+str(20)+'_'+str(n_filters)+'_'+ str(1e-4)+'_'+str(cnumber)+'_'+ str(depth)+'_'+str(loss_)+'_epochs=15'+'_'+'_model_2')
    


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, 256, 256, 2) 0                                            
__________________________________________________________________________________________________
conv2d_116 (Conv2D)             (None, 256, 256, 8)  152         input_17[0][0]                   
__________________________________________________________________________________________________
conv2d_117 (Conv2D)             (None, 256, 256, 8)  584         conv2d_116[0][0]                 
__________________________________________________________________________________________________
max_pooling2d_24 (MaxPooling2D) (None, 128, 128, 8)  0           conv2d_117[0][0]                 
____________________________________________________________________________________________

Epoch 1/15
20/20 [==============================] - 108s 5s/step - loss: 0.2069 - accuracy: 0.4866 - val_loss: 0.1462 - val_accuracy: 0.6031
Epoch 2/15
20/20 [==============================] - 106s 5s/step - loss: 0.1195 - accuracy: 0.7390 - val_loss: 0.0998 - val_accuracy: 0.7960
Epoch 3/15
20/20 [==============================] - 107s 5s/step - loss: 0.1013 - accuracy: 0.7951 - val_loss: 0.0945 - val_accuracy: 0.8090
Epoch 4/15
20/20 [==============================] - 92s 5s/step - loss: 0.0935 - accuracy: 0.8076 - val_loss: 0.0942 - val_accuracy: 0.8083
Epoch 5/15
20/20 [==============================] - 93s 5s/step - loss: 0.0946 - accuracy: 0.8065 - val_loss: 0.0894 - val_accuracy: 0.8174
Epoch 6/15
20/20 [==============================] - 95s 5s/step - loss: 0.0910 - accuracy: 0.8093 - val_loss: 0.0974 - val_accuracy: 0.8026
Epoch 7/15
20/20 [==============================] - 91s 5s/step - loss: 0.0911 - accuracy: 0.8127 - val_loss: 0.0889 - val_accuracy: 0.8185
Epoch 8/15
20/20 

Epoch 1/15
20/20 [==============================] - 95s 5s/step - loss: 0.2216 - accuracy: 0.5571 - val_loss: 0.2180 - val_accuracy: 0.5641
Epoch 2/15
20/20 [==============================] - 94s 5s/step - loss: 0.2212 - accuracy: 0.5575 - val_loss: 0.2180 - val_accuracy: 0.5641
Epoch 3/15
20/20 [==============================] - 92s 5s/step - loss: 0.2212 - accuracy: 0.5575 - val_loss: 0.2179 - val_accuracy: 0.5641
Epoch 4/15
20/20 [==============================] - 88s 4s/step - loss: 0.2212 - accuracy: 0.5576 - val_loss: 0.2179 - val_accuracy: 0.5641
Epoch 5/15
20/20 [==============================] - 94s 5s/step - loss: 0.2212 - accuracy: 0.5576 - val_loss: 0.2179 - val_accuracy: 0.5641
Epoch 6/15
20/20 [==============================] - 94s 5s/step - loss: 0.2212 - accuracy: 0.5576 - val_loss: 0.2179 - val_accuracy: 0.5642
Epoch 7/15
20/20 [==============================] - 94s 5s/step - loss: 0.2212 - accuracy: 0.5576 - val_loss: 0.2179 - val_accuracy: 0.5642
Epoch 8/15
20/20 [==

Epoch 1/15
20/20 [==============================] - 94s 5s/step - loss: 1.2478 - accuracy: 0.4493 - val_loss: 1.0616 - val_accuracy: 0.4203
Epoch 2/15
20/20 [==============================] - 93s 5s/step - loss: 0.9768 - accuracy: 0.5357 - val_loss: 0.8655 - val_accuracy: 0.6970
Epoch 3/15
20/20 [==============================] - 91s 5s/step - loss: 0.7993 - accuracy: 0.7451 - val_loss: 0.7490 - val_accuracy: 0.7743
Epoch 4/15
20/20 [==============================] - 93s 5s/step - loss: 0.7310 - accuracy: 0.7779 - val_loss: 0.7047 - val_accuracy: 0.7827
Epoch 5/15
20/20 [==============================] - 94s 5s/step - loss: 0.7082 - accuracy: 0.7791 - val_loss: 0.6880 - val_accuracy: 0.7873
Epoch 6/15
20/20 [==============================] - 96s 5s/step - loss: 0.6769 - accuracy: 0.7853 - val_loss: 0.6685 - val_accuracy: 0.7763
Epoch 7/15
20/20 [==============================] - 101s 5s/step - loss: 0.6594 - accuracy: 0.7847 - val_loss: 0.6357 - val_accuracy: 0.7975
Epoch 8/15
20/20 [=

Epoch 1/15
20/20 [==============================] - 109s 6s/step - loss: 0.9715 - accuracy: 0.3489 - val_loss: 0.3624 - val_accuracy: 0.3871
Epoch 2/15
20/20 [==============================] - 110s 6s/step - loss: 0.3264 - accuracy: 0.4408 - val_loss: 0.3058 - val_accuracy: 0.5721
Epoch 3/15
20/20 [==============================] - 116s 6s/step - loss: 0.2968 - accuracy: 0.5490 - val_loss: 0.2880 - val_accuracy: 0.5715
Epoch 4/15
20/20 [==============================] - 105s 5s/step - loss: 0.2825 - accuracy: 0.5785 - val_loss: 0.2762 - val_accuracy: 0.5560
Epoch 5/15
20/20 [==============================] - 94s 5s/step - loss: 0.2723 - accuracy: 0.5961 - val_loss: 0.2660 - val_accuracy: 0.5813
Epoch 6/15
20/20 [==============================] - 105s 5s/step - loss: 0.2614 - accuracy: 0.6023 - val_loss: 0.2585 - val_accuracy: 0.5613
Epoch 7/15
20/20 [==============================] - 103s 5s/step - loss: 0.2538 - accuracy: 0.6040 - val_loss: 0.2517 - val_accuracy: 0.6459
Epoch 8/15
20/